# 6. Tuning

In [1]:
MODEL = 'EBR'
TUNING_MODEL = 'jiaocheng'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
import os
import pickle
py_file_location = "./drive/My Drive/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

py_file_location = "./drive/My Drive/LAB/BrownlowH2H/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

import NingXiang

import pandas as pd

In [4]:
ningxiang = NingXiang.NingXiang()

NingXiang Initialised


Read in and Prepare Data

In [5]:
train_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/train_data.parquet')
val_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/val_data.parquet')
test_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/test_data.parquet')

In [6]:
train_other_info = train_data[['game_id', 'player1', 'player2']]
val_other_info = val_data[['game_id', 'player1', 'player2']]
test_other_info = test_data[['game_id', 'player1', 'player2']]

In [7]:
train_x = train_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
train_y = train_data['Brownlow Votes']
val_x = val_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
val_y = val_data['Brownlow Votes']
test_x = test_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
test_y = test_data['Brownlow Votes']

In [8]:
# ningxiang.read_in_train_data(train_x, train_y)
# ningxiang.set_model_type("Regression")
# ningxiang_output = ningxiang.get_rf_based_feature_combinations(3)

# with open(f'./drive/My Drive/LAB/BrownlowH2H/models/NingXing_output.pickle', 'rb') as f:
#     ningxiang_output = pickle.load(f)

In [9]:
with open('./drive/My Drive/LAB/BrownlowH2H/models/H2H_NingXiang.pickle', 'rb') as f:
  ningxiang_output = pickle.load(f)

Set parameters

In [10]:
import JiaoCheng_H2H

In [11]:
jiaocheng = JiaoCheng_H2H.JiaoCheng()

JiaoCheng Initialised


In [12]:
!pip install interpret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.0/758.0 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.0 MB/s eta 0

In [13]:
from interpret.glassbox import ExplainableBoostingRegressor as EBR

In [14]:
jiaocheng.read_in_model(EBR, 'Regression')

Successfully read in model <class 'interpret.glassbox._ebm._ebm.ExplainableBoostingRegressor'>, which is a Regression model


In [15]:
jiaocheng.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data


In [16]:
parameter_choices = {
    'max_bins': (4, 6, 64, 256, 1024, 4096),
    'min_samples_leaf': (2, 4, 8, 16, 32),
    'interactions': (2, 4, 8),
    'max_leaves': (100, 1000, 10000),
    'learning_rate': (0.001, 0.01, 0.1, 1),
    'max_rounds': (25, 50, 100, 200),

}

In [17]:
jiaocheng.set_hyperparameters(parameter_choices)

Successfully recorded hyperparameter choices


In [18]:
jiaocheng.set_non_tuneable_hyperparameters({'random_state': 19260817, 'n_jobs': -1})

Successfully recorded non_tuneable_hyperparameter choices


In [19]:
jiaocheng.set_features(ningxiang_output)

Successfully recorded tuneable feature combination choices and updated relevant internal structures


In [20]:
jiaocheng.set_tuning_order(['features', 'interactions', 'max_bins', 'max_leaves', 'min_samples_leaf', 'max_rounds', 'learning_rate'])

In [21]:
jiaocheng.set_hyperparameter_default_values({
    'features': 0,
    'max_bins': 1024,
    'min_samples_leaf': 4,
    'interactions': 8,
    'max_leaves': 1000,
    'learning_rate': 0.1,
    'max_rounds': 50})

In [22]:
try:
  jiaocheng.read_in_tuning_result_df(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv')
  print('True')
except:
  print('False')

Successfully read in tuning result of 103 rows
True


In [23]:
jiaocheng.set_tuning_result_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv')

Successfully set tuning output address


In [24]:
jiaocheng.set_best_model_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tmp_models/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')

Successfully set best model output address


In [25]:
jiaocheng.train_other_info = train_other_info
jiaocheng.val_other_info = val_other_info
jiaocheng.test_other_info = test_other_info

In [26]:
import warnings
warnings.filterwarnings("ignore")

In [27]:
jiaocheng.tune()


Default combo: [4, 1, 2, 1, 2, 1, 0] 


ROUND 1

Round 2 
Hyperparameter: features (index: 6) 

As new Best Combo (4, 1, 2, 1, 2, 1, 0) was read in, best_clf is set to None
Already Trained and Tested combination (4, 1, 2, 1, 2, 1, 0), which had val score of 0.5765
        Current best combo: (4, 1, 2, 1, 2, 1, 0) with val score 0.5765. 
        Has trained 103 of 155520 combinations so far
Already Trained and Tested combination (4, 1, 2, 1, 2, 1, 1), which had val score of 0.5744
        Current best combo: (4, 1, 2, 1, 2, 1, 0) with val score 0.5765. 
        Has trained 103 of 155520 combinations so far
Already Trained and Tested combination (4, 1, 2, 1, 2, 1, 2), which had val score of 0.5671
        Current best combo: (4, 1, 2, 1, 2, 1, 0) with val score 0.5765. 
        Has trained 103 of 155520 combinations so far
As new Best Combo (4, 1, 2, 1, 2, 1, 3) was read in, best_clf is set to None
Already Trained and Tested combination (4, 1, 2, 1, 2, 1, 3), which had val score of 0.59

In [28]:
jiaocheng.view_best_combo_and_score()

Max Score: 
 0.609015
Max Test Score: 
 0.609015
Max Combo Index: 
 (2, 1, 2, 1, 2, 3, 3) out of [6, 5, 3, 3, 4, 4, 36] (note best combo is 0-indexed)
Max Combo Hyperparamer Combination: 
 {'max_bins': 64, 'min_samples_leaf': 4, 'interactions': 8, 'max_leaves': 1000, 'learning_rate': 0.1, 'max_rounds': 200, 'features': 3}
Max Combo Features: 
 ('AFLCA_votes', 'Disposals', 'Contested Possessions', 'Goals', 'Centre Clearances', 'Intercepts')
% Combos Checked: 103 out of 155520 which is 0.066229%
